In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import grid, figure, plot, savefig
import matplotlib.pyplot as plt
import os
import time

In [167]:

lat = 42.46386370313852 #shitot ot ecvat   MARKS   TPLK
lon = 78.52905690246126# dolg merid-i po ekvat
ls_files =[]
test = []
# Указываем папку в которой производить поиск
path='./'
for rootdir, dirs, files in os.walk(path):
    for file in files:       
        if((file.split('.')[-1])=='dat'):
            ls_files.append(file)
            #print (os.path.join(rootdir, file))
            

cnt_file = len(ls_files)
for i in range(cnt_file):
    t=[]
    #test.append(i)
    file1 = open (ls_files[i])
    #print(file1)
    allLines = file1.readlines()
    file1linecount = len(allLines)
    file1.seek(0) # peremestit ukozatel v nachal

    for ln in range(file1linecount) :#3700 strok все строки
    #for ln in range(file1) :#3700 strok все строки
        line  = file1.readline()
        if ln > 10 :
            param=line.strip() #str
            param = param.split()
           # print(param)
            t.append(param)
        #np.array(param)
    test.append(t)
arr = np.array(test)
arr = arr.astype(float)
cnt =15
G5 = pd.DataFrame(arr[0],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G5['Cr']=  G5['code']-G5['phase'] 
G5['Coef']= ((1/G5['Cr'].count())*G5['Cr'].sum())
G5['sTEC']= G5['phase']+G5['Coef']

G5['vTEC'] =  G5['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G5['el']*3.14/180))  /  (6400+350) )  )     ) #
G5['psi'] = np.pi/2 - np.radians(G5['el'])- np.arcsin(6371 * np.cos( np.radians(G5['el']) )/(6371+350) ) #1.209
G5['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G5['psi']) + np.cos(np.radians(lat))*np.sin(G5['psi']) *np.cos(np.radians(G5['az'])) ))
G5['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G5['psi'] ) * np.sin(np.radians( G5['az'] ) ) * (1/np.cos( np.radians( G5['lat'] )) ) )  ) 
G5['Dlat'] = np.radians(G5['lat']) - np.radians(lat)
G5['Dlon'] = np.radians(G5['lon']) - np.radians(lon)
G5['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G5['el']))/(6371+350) )    )**-1 )# factor naklona
G5 = G5[0:50]
G5['t']= G5['hour'].iloc[cnt] - G5['hour']
#G5['S1'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G5['el']))/(6371+350) )    ) )# factor naklona
#G5['w'] = 1/G5['S'] *(  1+ ((2* G5['t'])/1)**2     )**-1 # ves =  delt t / interval 
G5['w'] = 1/G5['S'] *(  1+ ((2* G5['t'])/(G5['hour'].iloc[-1]-G5['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G5.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G5['w*S']=      (G5['w'] * G5['S']) 
G5['w*S*lat']=  (G5['w'] * G5['S'] * G5['Dlat'])
G5['w*S*lat^2']=(G5['w'] * G5['S'] * G5['Dlat']**2)
G5['w*S*lon']=  (G5['w'] * G5['S'] * G5['Dlon'])
G5['w*S*lon^2']=(G5['w'] * G5['S'] * G5['Dlon']**2)
G5['w*S*t']=    (G5['w'] * G5['S'] * G5['t'])
G5['w*S*t^2']=  (G5['w'] * G5['S'] * G5['t']**2)
G5['w*I']=       G5['w'] * G5['sTEC']
G5['w*S*I'] =       G5['w'] * G5['S']* G5['sTEC']             
G5['w*S*I*lat']=    G5['w'] * G5['S']* G5['sTEC'] * G5['Dlat']
G5['w*S*I*lat^2']=  G5['w'] * G5['S']* G5['sTEC'] * G5['Dlat']**2
G5['w*S*I*lon']=    G5['w'] * G5['S']* G5['sTEC'] * G5['Dlon']
G5['w*S*I*lon^2']=  G5['w'] * G5['S']* G5['sTEC'] * G5['Dlon']**2
G5['w*S*I*t']=      G5['w'] * G5['S']* G5['sTEC'] * G5['t']
G5['w*S*I*t^2']=    G5['w'] * G5['S']* G5['sTEC'] * G5['t']**2
G5['(w)']=         G5['w'].sum()#              A4
G5['(w*I)'] = G5['w*I'].sum()
G5['(w*S)'] = G5['w*S'].sum()
G5['(w*S*lat)'] = G5['w*S*lat'].sum()#              A2 A3
G5['(w*S*lat^2)'] = G5['w*S*lat^2'].sum()#            A2 A3
G5['(w*S*lon)']= G5['w*S*lon'].sum()#            A2 A3
G5['(w*S*lon^2)']= (G5['w*S*lon^2']).sum()#            A2 A3
G5['(w*S*t)']= (G5['w*S*t']).sum()#            A2 A3
G5['(w*S*t^2)']= (G5['w*S*t^2']).sum()#            A2 A3
G5['w*S^2']=         G5['w'] * G5['S']**2
G5['w*S^2*lat']  =   G5['w'] * G5['S']**2 * G5['Dlat']
G5['w*S^2*lat^2']  = G5['w'] * G5['S']**2 * G5['Dlat']**2
G5['w*S^2*lat^3']  = G5['w'] * G5['S']**2 * G5['Dlat']**3
G5['w*S^2*lat^4']  = G5['w'] * G5['S']**2 * G5['Dlat']**4
G5['w*S^2*lon']  =   G5['w'] * G5['S']**2 * G5['Dlon']
G5['w*S^2*lon^2']  = G5['w'] * G5['S']**2 * G5['Dlon']**2
G5['w*S^2*lon^3']  = G5['w'] * G5['S']**2 * G5['Dlon']**3
G5['w*S^2*lon^4']  = G5['w'] * G5['S']**2 * G5['Dlon']**4
G5['w*S^2*t']  =     G5['w'] * G5['S']**2 * G5['t']
G5['w*S^2*t^2']  =   G5['w'] * G5['S']**2 * G5['t']**2
G5['w*S^2*t^3']  =   G5['w'] * G5['S']**2 * G5['t']**3
G5['w*S^2*t^4']  =   G5['w'] * G5['S']**2 * G5['t']**4
G5['w*S^2*lat*lon']  =   G5['w'] * G5['S']**2 * G5['Dlat']* G5['Dlon']
G5['w*S^2*lat^2*lon']  = G5['w'] * G5['S']**2 * G5['Dlat']**2 * G5['Dlon']
G5['w*S^2*lat*lon^2']  = G5['w'] * G5['S']**2 * G5['Dlat']* G5['Dlon']**2
G5['w*S^2*lat^2*lon^2']= G5['w'] * G5['S']**2 * G5['Dlat']**2 * G5['Dlon']**2
G5['w*S^2*lat*t']  =     G5['w'] * G5['S']**2 * G5['Dlat']* G5['t']
G5['w*S^2*lat^2*t']  =   G5['w'] * G5['S']**2 * G5['Dlat']**2 * G5['t']
G5['w*S^2*lat*t^2']  =   G5['w'] * G5['S']**2 * G5['Dlat']* G5['t']**2
G5['w*S^2*lat^2*t^2']=   G5['w'] * G5['S']**2 * G5['Dlat']**2 * G5['t']**2
G5['w*S^2*lon*t']  =     G5['w'] * G5['S']**2 * G5['Dlon']* G5['t']
G5['w*S^2*lon^2*t']  =   G5['w'] * G5['S']**2 * G5['Dlon']**2 * G5['t']
G5['w*S^2*lon*t^2']  =   G5['w'] * G5['S']**2 * G5['Dlon']* G5['t']**2
G5['w*S^2*lon^2*t^2']=   G5['w'] * G5['S']**2 * G5['Dlon']**2 * G5['t']**2
#G5.to_csv('G5.csv',sep=';')


G10 = pd.DataFrame(arr[1],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G10['Cr']=  G10['code']-G10['phase'] 
G10['Coef']= ((1/G10['Cr'].count())*G10['Cr'].sum())
G10['sTEC']= G10['phase']+G10['Coef']

G10['vTEC'] =  G10['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G10['el']*3.14/180))  /  (6400+350) )  )     ) #
G10['psi'] = np.pi/2 - np.radians(G10['el'])- np.arcsin(6371 * np.cos( np.radians(G10['el']) )/(6371+350) ) #1.209
G10['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G10['psi']) + np.cos(np.radians(lat))*np.sin(G10['psi']) *np.cos(np.radians(G10['az'])) ))
G10['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G10['psi'] ) * np.sin(np.radians( G10['az'] ) ) * (1/np.cos( np.radians( G10['lat'] )) ) )  ) 
G10['Dlat'] = np.radians(G10['lat']) - np.radians(lat)
G10['Dlon'] = np.radians(G10['lon']) - np.radians(lon)
G10['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G10['el']))/(6371+350) )    )**-1 )# factor naklona
G10 = G10[0:50]
G10['t']= G10['hour'].iloc[cnt]          - G10['hour']
G10['w'] = 1/G10['S'] *(  1+ ((2* G10['t'])/(G10['hour'].iloc[-1]-G10['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G10.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G10['w*S']=      (G10['w'] * G10['S']) 
G10['w*S*lat']=  (G10['w'] * G10['S'] * G10['Dlat'])
G10['w*S*lat^2']=(G10['w'] * G10['S'] * G10['Dlat']**2)
G10['w*S*lon']=  (G10['w'] * G10['S'] * G10['Dlon'])
G10['w*S*lon^2']=(G10['w'] * G10['S'] * G10['Dlon']**2)
G10['w*S*t']=    (G10['w'] * G10['S'] * G10['t'])
G10['w*S*t^2']=  (G10['w'] * G10['S'] * G10['t']**2)
G10['w*I']=       G10['w'] * G10['sTEC']
G10['w*S*I'] =       G10['w'] * G10['S']* G10['sTEC']             
G10['w*S*I*lat']=    G10['w'] * G10['S']* G10['sTEC'] * G10['Dlat']
G10['w*S*I*lat^2']=  G10['w'] * G10['S']* G10['sTEC'] * G10['Dlat']**2
G10['w*S*I*lon']=    G10['w'] * G10['S']* G10['sTEC'] * G10['Dlon']
G10['w*S*I*lon^2']=  G10['w'] * G10['S']* G10['sTEC'] * G10['Dlon']**2
G10['w*S*I*t']=      G10['w'] * G10['S']* G10['sTEC'] * G10['t']
G10['w*S*I*t^2']=    G10['w'] * G10['S']* G10['sTEC'] * G10['t']**2
G10['(w)']=         G10['w'].sum()#              A4
G10['(w*I)'] = G10['w*I'].sum()
G10['(w*S)'] =G10['w*S'].sum()
G10['(w*S*lat)'] = G10['w*S*lat'].sum()#              A2 A3
G10['(w*S*lat^2)'] = G10['w*S*lat^2'].sum()#            A2 A3
G10['(w*S*lon)']= G10['w*S*lon'].sum()#            A2 A3
G10['(w*S*lon^2)']= (G10['w*S*lon^2']).sum()#            A2 A3
G10['(w*S*t)']= (G10['w*S*t']).sum()#            A2 A3
G10['(w*S*t^2)']= (G10['w*S*t^2']).sum()#            A2 A3
G10['w*S^2']=         G10['w'] * G10['S']**2
G10['w*S^2*lat']  =   G10['w'] * G10['S']**2 * G10['Dlat']
G10['w*S^2*lat^2']  = G10['w'] * G10['S']**2 * G10['Dlat']**2
G10['w*S^2*lat^3']  = G10['w'] * G10['S']**2 * G10['Dlat']**3
G10['w*S^2*lat^4']  = G10['w'] * G10['S']**2 * G10['Dlat']**4
G10['w*S^2*lon']  =   G10['w'] * G10['S']**2 * G10['Dlon']
G10['w*S^2*lon^2']  = G10['w'] * G10['S']**2 * G10['Dlon']**2
G10['w*S^2*lon^3']  = G10['w'] * G10['S']**2 * G10['Dlon']**3
G10['w*S^2*lon^4']  = G10['w'] * G10['S']**2 * G10['Dlon']**4
G10['w*S^2*t']  =     G10['w'] * G10['S']**2 * G10['t']
G10['w*S^2*t^2']  =   G10['w'] * G10['S']**2 * G10['t']**2
G10['w*S^2*t^3']  =   G10['w'] * G10['S']**2 * G10['t']**3
G10['w*S^2*t^4']  =   G10['w'] * G10['S']**2 * G10['t']**4
G10['w*S^2*lat*lon']  =   G10['w'] * G10['S']**2 * G10['Dlat']* G10['Dlon']
G10['w*S^2*lat^2*lon']  = G10['w'] * G10['S']**2 * G10['Dlat']**2 * G10['Dlon']
G10['w*S^2*lat*lon^2']  = G10['w'] * G10['S']**2 * G10['Dlat']* G10['Dlon']**2
G10['w*S^2*lat^2*lon^2']= G10['w'] * G10['S']**2 * G10['Dlat']**2 * G10['Dlon']**2
G10['w*S^2*lat*t']  =     G10['w'] * G10['S']**2 * G10['Dlat']* G10['t']
G10['w*S^2*lat^2*t']  =   G10['w'] * G10['S']**2 * G10['Dlat']**2 * G10['t']
G10['w*S^2*lat*t^2']  =   G10['w'] * G10['S']**2 * G10['Dlat']* G10['t']**2
G10['w*S^2*lat^2*t^2']=   G10['w'] * G10['S']**2 * G10['Dlat']**2 * G10['t']**2
G10['w*S^2*lon*t']  =     G10['w'] * G10['S']**2 * G10['Dlon']* G10['t']
G10['w*S^2*lon^2*t']  =   G10['w'] * G10['S']**2 * G10['Dlon']**2 * G10['t']
G10['w*S^2*lon*t^2']  =   G10['w'] * G10['S']**2 * G10['Dlon']* G10['t']**2
G10['w*S^2*lon^2*t^2']=   G10['w'] * G10['S']**2 * G10['Dlon']**2 * G10['t']**2

G13 = pd.DataFrame(arr[2],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G13['Cr']=  G13['code']-G13['phase'] 
G13['Coef']= ((1/G13['Cr'].count())*G13['Cr'].sum())
G13['sTEC']= G13['phase']+G13['Coef']

G13['vTEC'] =  G13['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G13['el']*3.14/180))  /  (6400+350) )  )     ) #
G13['psi'] = np.pi/2 - np.radians(G13['el'])- np.arcsin(6371 * np.cos( np.radians(G13['el']) )/(6371+350) ) #1.209
G13['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G13['psi']) + np.cos(np.radians(lat))*np.sin(G13['psi']) *np.cos(np.radians(G13['az'])) ))
G13['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G13['psi'] ) * np.sin(np.radians( G13['az'] ) ) * (1/np.cos( np.radians( G13['lat'] )) ) )  ) 
G13['Dlat'] = np.radians(G13['lat']) - np.radians(lat)
G13['Dlon'] = np.radians(G13['lon']) - np.radians(lon)
G13['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G13['el']))/(6371+350) )    )**-1 )# factor naklona
G13 = G13[0:50]
G13['t']= G13['hour'].iloc[cnt]          - G13['hour']
G13['w'] = 1/G13['S'] *(  1+ ((2* G13['t'])/(G13['hour'].iloc[-1]-G13['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G13.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G13['w*S']=      (G13['w'] * G13['S']) 
G13['w*S*lat']=  (G13['w'] * G13['S'] * G13['Dlat'])
G13['w*S*lat^2']=(G13['w'] * G13['S'] * G13['Dlat']**2)
G13['w*S*lon']=  (G13['w'] * G13['S'] * G13['Dlon'])
G13['w*S*lon^2']=(G13['w'] * G13['S'] * G13['Dlon']**2)
G13['w*S*t']=    (G13['w'] * G13['S'] * G13['t'])
G13['w*S*t^2']=  (G13['w'] * G13['S'] * G13['t']**2)
G13['w*I']=       G13['w'] * G13['sTEC']
G13['w*S*I'] =       G13['w'] * G13['S']* G13['sTEC']             
G13['w*S*I*lat']=    G13['w'] * G13['S']* G13['sTEC'] * G13['Dlat']
G13['w*S*I*lat^2']=  G13['w'] * G13['S']* G13['sTEC'] * G13['Dlat']**2
G13['w*S*I*lon']=    G13['w'] * G13['S']* G13['sTEC'] * G13['Dlon']
G13['w*S*I*lon^2']=  G13['w'] * G13['S']* G13['sTEC'] * G13['Dlon']**2
G13['w*S*I*t']=      G13['w'] * G13['S']* G13['sTEC'] * G13['t']
G13['w*S*I*t^2']=    G13['w'] * G13['S']* G13['sTEC'] * G13['t']**2
G13['(w)']=         G13['w'].sum()#              A4
G13['(w*I)'] = G13['w*I'].sum()
G13['(w*S)']=G13['w*S'].sum()
G13['(w*S*lat)'] = G13['w*S*lat'].sum()#              A2 A3
G13['(w*S*lat^2)'] = G13['w*S*lat^2'].sum()#            A2 A3
G13['(w*S*lon)']= G13['w*S*lon'].sum()#            A2 A3
G13['(w*S*lon^2)']= (G13['w*S*lon^2']).sum()#            A2 A3
G13['(w*S*t)']= (G13['w*S*t']).sum()#            A2 A3
G13['(w*S*t^2)']= (G13['w*S*t^2']).sum()#            A2 A3
G13['w*S^2']=         G13['w'] * G13['S']**2
G13['w*S^2*lat']  =   G13['w'] * G13['S']**2 * G13['Dlat']
G13['w*S^2*lat^2']  = G13['w'] * G13['S']**2 * G13['Dlat']**2
G13['w*S^2*lat^3']  = G13['w'] * G13['S']**2 * G13['Dlat']**3
G13['w*S^2*lat^4']  = G13['w'] * G13['S']**2 * G13['Dlat']**4
G13['w*S^2*lon']  =   G13['w'] * G13['S']**2 * G13['Dlon']
G13['w*S^2*lon^2']  = G13['w'] * G13['S']**2 * G13['Dlon']**2
G13['w*S^2*lon^3']  = G13['w'] * G13['S']**2 * G13['Dlon']**3
G13['w*S^2*lon^4']  = G13['w'] * G13['S']**2 * G13['Dlon']**4
G13['w*S^2*t']  =     G13['w'] * G13['S']**2 * G13['t']
G13['w*S^2*t^2']  =   G13['w'] * G13['S']**2 * G13['t']**2
G13['w*S^2*t^3']  =   G13['w'] * G13['S']**2 * G13['t']**3
G13['w*S^2*t^4']  =   G13['w'] * G13['S']**2 * G13['t']**4
G13['w*S^2*lat*lon']  =   G13['w'] * G13['S']**2 * G13['Dlat']* G13['Dlon']
G13['w*S^2*lat^2*lon']  = G13['w'] * G13['S']**2 * G13['Dlat']**2 * G13['Dlon']
G13['w*S^2*lat*lon^2']  = G13['w'] * G13['S']**2 * G13['Dlat']* G13['Dlon']**2
G13['w*S^2*lat^2*lon^2']= G13['w'] * G13['S']**2 * G13['Dlat']**2 * G13['Dlon']**2
G13['w*S^2*lat*t']  =     G13['w'] * G13['S']**2 * G13['Dlat']* G13['t']
G13['w*S^2*lat^2*t']  =   G13['w'] * G13['S']**2 * G13['Dlat']**2 * G13['t']
G13['w*S^2*lat*t^2']  =   G13['w'] * G13['S']**2 * G13['Dlat']* G13['t']**2
G13['w*S^2*lat^2*t^2']=   G13['w'] * G13['S']**2 * G13['Dlat']**2 * G13['t']**2
G13['w*S^2*lon*t']  =     G13['w'] * G13['S']**2 * G13['Dlon']* G13['t']
G13['w*S^2*lon^2*t']  =   G13['w'] * G13['S']**2 * G13['Dlon']**2 * G13['t']
G13['w*S^2*lon*t^2']  =   G13['w'] * G13['S']**2 * G13['Dlon']* G13['t']**2
G13['w*S^2*lon^2*t^2']=   G13['w'] * G13['S']**2 * G13['Dlon']**2 * G13['t']**2

G15 = pd.DataFrame(arr[3],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G15['Cr']=  G15['code']-G15['phase'] 
G15['Coef']= ((1/G15['Cr'].count())*G15['Cr'].sum())
G15['sTEC']= G15['phase']+G15['Coef']
#G15.plot.scatter(x='hour',y='sTEC',c='DarkBlue')
G15['vTEC'] =  G15['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G15['el']*3.14/180))  /  (6400+350) )  )     ) #
G15['psi'] = np.pi/2 - np.radians(G15['el'])- np.arcsin(6371 * np.cos( np.radians(G15['el']) )/(6371+350) ) #1.209
G15['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G15['psi']) + np.cos(np.radians(lat))*np.sin(G15['psi']) *np.cos(np.radians(G15['az'])) ))
G15['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G15['psi'] ) * np.sin(np.radians( G15['az'] ) ) * (1/np.cos( np.radians( G15['lat'] )) ) )  ) 
G15['Dlat'] = np.radians(G15['lat']) - np.radians(lat)
G15['Dlon'] = np.radians(G15['lon']) - np.radians(lon)
G15['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G15['el']))/(6371+350) )    )**-1 )# factor naklona
G15 = G15[0:50]
G15['t']= G15['hour'].iloc[cnt]          - G15['hour']
G15['w'] = 1/G15['S'] *(  1+ ((2* G15['t'])/(G15['hour'].iloc[-1]-G15['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G15.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G15['w*S']=      (G15['w'] * G15['S']) 
G15['w*S*lat']=  (G15['w'] * G15['S'] * G15['Dlat'])
G15['w*S*lat^2']=(G15['w'] * G15['S'] * G15['Dlat']**2)
G15['w*S*lon']=  (G15['w'] * G15['S'] * G15['Dlon'])
G15['w*S*lon^2']=(G15['w'] * G15['S'] * G15['Dlon']**2)
G15['w*S*t']=    (G15['w'] * G15['S'] * G15['t'])
G15['w*S*t^2']=  (G15['w'] * G15['S'] * G15['t']**2)
G15['w*I']=       G15['w'] * G15['sTEC']
G15['w*S*I'] =       G15['w'] * G15['S']* G15['sTEC']             
G15['w*S*I*lat']=    G15['w'] * G15['S']* G15['sTEC'] * G15['Dlat']
G15['w*S*I*lat^2']=  G15['w'] * G15['S']* G15['sTEC'] * G15['Dlat']**2
G15['w*S*I*lon']=    G15['w'] * G15['S']* G15['sTEC'] * G15['Dlon']
G15['w*S*I*lon^2']=  G15['w'] * G15['S']* G15['sTEC'] * G15['Dlon']**2
G15['w*S*I*t']=      G15['w'] * G15['S']* G15['sTEC'] * G15['t']
G15['w*S*I*t^2']=    G15['w'] * G15['S']* G15['sTEC'] * G15['t']**2
G15['(w)']=         G15['w'].sum()#              A4
G15['(w*I)'] = G15['w*I'].sum()
G15['(w*S)'] = G15['w*S'].sum()
G15['(w*S*lat)'] = G15['w*S*lat'].sum()#              A2 A3
G15['(w*S*lat^2)'] = G15['w*S*lat^2'].sum()#            A2 A3
G15['(w*S*lon)']= G15['w*S*lon'].sum()#            A2 A3
G15['(w*S*lon^2)']= (G15['w*S*lon^2']).sum()#            A2 A3
G15['(w*S*t)']= (G15['w*S*t']).sum()#            A2 A3
G15['(w*S*t^2)']= (G15['w*S*t^2']).sum()#            A2 A3
G15['w*S^2']=         G15['w'] * G15['S']**2
G15['w*S^2*lat']  =   G15['w'] * G15['S']**2 * G15['Dlat']
G15['w*S^2*lat^2']  = G15['w'] * G15['S']**2 * G15['Dlat']**2
G15['w*S^2*lat^3']  = G15['w'] * G15['S']**2 * G15['Dlat']**3
G15['w*S^2*lat^4']  = G15['w'] * G15['S']**2 * G15['Dlat']**4
G15['w*S^2*lon']  =   G15['w'] * G15['S']**2 * G15['Dlon']
G15['w*S^2*lon^2']  = G15['w'] * G15['S']**2 * G15['Dlon']**2
G15['w*S^2*lon^3']  = G15['w'] * G15['S']**2 * G15['Dlon']**3
G15['w*S^2*lon^4']  = G15['w'] * G15['S']**2 * G15['Dlon']**4
G15['w*S^2*t']  =     G15['w'] * G15['S']**2 * G15['t']
G15['w*S^2*t^2']  =   G15['w'] * G15['S']**2 * G15['t']**2
G15['w*S^2*t^3']  =   G15['w'] * G15['S']**2 * G15['t']**3
G15['w*S^2*t^4']  =   G15['w'] * G15['S']**2 * G15['t']**4
G15['w*S^2*lat*lon']  =   G15['w'] * G15['S']**2 * G15['Dlat']* G15['Dlon']
G15['w*S^2*lat^2*lon']  = G15['w'] * G15['S']**2 * G15['Dlat']**2 * G15['Dlon']
G15['w*S^2*lat*lon^2']  = G15['w'] * G15['S']**2 * G15['Dlat']* G15['Dlon']**2
G15['w*S^2*lat^2*lon^2']= G15['w'] * G15['S']**2 * G15['Dlat']**2 * G15['Dlon']**2
G15['w*S^2*lat*t']  =     G15['w'] * G15['S']**2 * G15['Dlat']* G15['t']
G15['w*S^2*lat^2*t']  =   G15['w'] * G15['S']**2 * G15['Dlat']**2 * G15['t']
G15['w*S^2*lat*t^2']  =   G15['w'] * G15['S']**2 * G15['Dlat']* G15['t']**2
G15['w*S^2*lat^2*t^2']=   G15['w'] * G15['S']**2 * G15['Dlat']**2 * G15['t']**2
G15['w*S^2*lon*t']  =     G15['w'] * G15['S']**2 * G15['Dlon']* G15['t']
G15['w*S^2*lon^2*t']  =   G15['w'] * G15['S']**2 * G15['Dlon']**2 * G15['t']
G15['w*S^2*lon*t^2']  =   G15['w'] * G15['S']**2 * G15['Dlon']* G15['t']**2
G15['w*S^2*lon^2*t^2']=   G15['w'] * G15['S']**2 * G15['Dlon']**2 * G15['t']**2

G18 = pd.DataFrame(arr[4],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G18['Cr']=  G18['code']-G18['phase'] 
G18['Coef']= ((1/G18['Cr'].count())*G18['Cr'].sum())
G18['sTEC']= G18['phase']+G18['Coef']
#G18.plot.scatter(x='hour',y='sTEC',c='DarkBlue')
G18['vTEC'] =  G18['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G18['el']*3.14/180))  /  (6400+350) )  )     ) #
G18['psi'] = np.pi/2 - np.radians(G18['el'])- np.arcsin(6371 * np.cos( np.radians(G18['el']) )/(6371+350) ) #1.209
G18['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G18['psi']) + np.cos(np.radians(lat))*np.sin(G18['psi']) *np.cos(np.radians(G18['az'])) ))
G18['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G18['psi'] ) * np.sin(np.radians( G18['az'] ) ) * (1/np.cos( np.radians( G18['lat'] )) ) )  ) 
G18['Dlat'] = np.radians(G18['lat']) - np.radians(lat)
G18['Dlon'] = np.radians(G18['lon']) - np.radians(lon)
G18['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G18['el']))/(6371+350) )    )**-1 )# factor naklona
G18 = G18[0:50]
G18['t']= G18['hour'].iloc[cnt]          - G18['hour']
G18['w'] = 1/G18['S'] *(  1+ ((2* G18['t'])/(G18['hour'].iloc[-1]-G18['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G18.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G18['w*S']=      (G18['w'] * G18['S']) 
G18['w*S*lat']=  (G18['w'] * G18['S'] * G18['Dlat'])
G18['w*S*lat^2']=(G18['w'] * G18['S'] * G18['Dlat']**2)
G18['w*S*lon']=  (G18['w'] * G18['S'] * G18['Dlon'])
G18['w*S*lon^2']=(G18['w'] * G18['S'] * G18['Dlon']**2)
G18['w*S*t']=    (G18['w'] * G18['S'] * G18['t'])
G18['w*S*t^2']=  (G18['w'] * G18['S'] * G18['t']**2)
G18['w*I']=       G18['w'] * G18['sTEC']
G18['w*S*I'] =       G18['w'] * G18['S']* G18['sTEC']             
G18['w*S*I*lat']=    G18['w'] * G18['S']* G18['sTEC'] * G18['Dlat']
G18['w*S*I*lat^2']=  G18['w'] * G18['S']* G18['sTEC'] * G18['Dlat']**2
G18['w*S*I*lon']=    G18['w'] * G18['S']* G18['sTEC'] * G18['Dlon']
G18['w*S*I*lon^2']=  G18['w'] * G18['S']* G18['sTEC'] * G18['Dlon']**2
G18['w*S*I*t']=      G18['w'] * G18['S']* G18['sTEC'] * G18['t']
G18['w*S*I*t^2']=    G18['w'] * G18['S']* G18['sTEC'] * G18['t']**2
G18['(w*S)'] = G18['w*S'].sum()
G18['(w)']=         G18['w'].sum()#              A4
G18['(w*I)'] = G18['w*I'].sum()
G18['(w*S*lat)'] = G18['w*S*lat'].sum()#              A2 A3
G18['(w*S*lat^2)'] = G18['w*S*lat^2'].sum()#            A2 A3
G18['(w*S*lon)']= G18['w*S*lon'].sum()#            A2 A3
G18['(w*S*lon^2)']= (G18['w*S*lon^2']).sum()#            A2 A3
G18['(w*S*t)']= (G18['w*S*t']).sum()#            A2 A3
G18['(w*S*t^2)']= (G18['w*S*t^2']).sum()#            A2 A3
G18['w*S^2']=         G18['w'] * G18['S']**2
G18['w*S^2*lat']  =   G18['w'] * G18['S']**2 * G18['Dlat']
G18['w*S^2*lat^2']  = G18['w'] * G18['S']**2 * G18['Dlat']**2
G18['w*S^2*lat^3']  = G18['w'] * G18['S']**2 * G18['Dlat']**3
G18['w*S^2*lat^4']  = G18['w'] * G18['S']**2 * G18['Dlat']**4
G18['w*S^2*lon']  =   G18['w'] * G18['S']**2 * G18['Dlon']
G18['w*S^2*lon^2']  = G18['w'] * G18['S']**2 * G18['Dlon']**2
G18['w*S^2*lon^3']  = G18['w'] * G18['S']**2 * G18['Dlon']**3
G18['w*S^2*lon^4']  = G18['w'] * G18['S']**2 * G18['Dlon']**4
G18['w*S^2*t']  =     G18['w'] * G18['S']**2 * G18['t']
G18['w*S^2*t^2']  =   G18['w'] * G18['S']**2 * G18['t']**2
G18['w*S^2*t^3']  =   G18['w'] * G18['S']**2 * G18['t']**3
G18['w*S^2*t^4']  =   G18['w'] * G18['S']**2 * G18['t']**4
G18['w*S^2*lat*lon']  =   G18['w'] * G18['S']**2 * G18['Dlat']* G18['Dlon']
G18['w*S^2*lat^2*lon']  = G18['w'] * G18['S']**2 * G18['Dlat']**2 * G18['Dlon']
G18['w*S^2*lat*lon^2']  = G18['w'] * G18['S']**2 * G18['Dlat']* G18['Dlon']**2
G18['w*S^2*lat^2*lon^2']= G18['w'] * G18['S']**2 * G18['Dlat']**2 * G18['Dlon']**2
G18['w*S^2*lat*t']  =     G18['w'] * G18['S']**2 * G18['Dlat']* G18['t']
G18['w*S^2*lat^2*t']  =   G18['w'] * G18['S']**2 * G18['Dlat']**2 * G18['t']
G18['w*S^2*lat*t^2']  =   G18['w'] * G18['S']**2 * G18['Dlat']* G18['t']**2
G18['w*S^2*lat^2*t^2']=   G18['w'] * G18['S']**2 * G18['Dlat']**2 * G18['t']**2
G18['w*S^2*lon*t']  =     G18['w'] * G18['S']**2 * G18['Dlon']* G18['t']
G18['w*S^2*lon^2*t']  =   G18['w'] * G18['S']**2 * G18['Dlon']**2 * G18['t']
G18['w*S^2*lon*t^2']  =   G18['w'] * G18['S']**2 * G18['Dlon']* G18['t']**2
G18['w*S^2*lon^2*t^2']=   G18['w'] * G18['S']**2 * G18['Dlon']**2 * G18['t']**2

G23 = pd.DataFrame(arr[5],  columns=['tsn', 'hour', 'el', 'az','phase','code','valid'])
G23['Cr']=  G23['code']-G23['phase'] 
G23['Coef']= ((1/G23['Cr'].count())*G23['Cr'].sum())
G23['sTEC']= G23['phase']+G23['Coef']
#G23.plot.scatter(x='hour',y='sTEC',c='DarkBlue')
G23['vTEC'] =  G23['sTEC']*  (np.cos  (np.arcsin(  (6400*np.cos(G23['el']*3.14/180))  /  (6400+350) )  )     ) #
G23['psi'] = np.pi/2 - np.radians(G23['el'])- np.arcsin(6371 * np.cos( np.radians(G23['el']) )/(6371+350) ) #1.209
G23['lat']=  np.degrees(np.arcsin( np.sin(np.radians(lat)) * np.cos(G23['psi']) + np.cos(np.radians(lat))*np.sin(G23['psi']) *np.cos(np.radians(G23['az'])) ))
G23['lon'] = np.degrees(np.radians(lon)+ np.arcsin( np.sin( G23['psi'] ) * np.sin(np.radians( G23['az'] ) ) * (1/np.cos( np.radians( G23['lat'] )) ) )  ) 
G23['Dlat'] = np.radians(G23['lat']) - np.radians(lat)
G23['Dlon'] = np.radians(G23['lon']) - np.radians(lon)
G23['S'] = (np.cos(  np.arcsin( 6371* np.sin(0.97* np.radians(90-G23['el']))/(6371+350) )    )**-1 )# factor naklona
G23 = G23[0:50]
G23['t']= G23['hour'].iloc[cnt]          - G23['hour']
G23['w'] = 1/G23['S'] *(  1+ ((2* G23['t'])/(G23['hour'].iloc[-1]-G23['hour'].iloc[0]))**2     )**-1 # ves =  delt t / interval 
G23.drop(['psi','valid','az','el','phase','code','Cr','Coef','lat','lon','tsn','hour','vTEC'], axis=1, inplace=True) 
G23['w*S']=      (G23['w'] * G23['S']) 
G23['w*S*lat']=  (G23['w'] * G23['S'] * G23['Dlat'])
G23['w*S*lat^2']=(G23['w'] * G23['S'] * G23['Dlat']**2)
G23['w*S*lon']=  (G23['w'] * G23['S'] * G23['Dlon'])
G23['w*S*lon^2']=(G23['w'] * G23['S'] * G23['Dlon']**2)
G23['w*S*t']=    (G23['w'] * G23['S'] * G23['t'])
G23['w*S*t^2']=  (G23['w'] * G23['S'] * G23['t']**2)
G23['w*I']=       G23['w'] * G23['sTEC']
G23['w*S*I'] =       G23['w'] * G23['S']* G23['sTEC']             
G23['w*S*I*lat']=    G23['w'] * G23['S']* G23['sTEC'] * G23['Dlat']
G23['w*S*I*lat^2']=  G23['w'] * G23['S']* G23['sTEC'] * G23['Dlat']**2
G23['w*S*I*lon']=    G23['w'] * G23['S']* G23['sTEC'] * G23['Dlon']
G23['w*S*I*lon^2']=  G23['w'] * G23['S']* G23['sTEC'] * G23['Dlon']**2
G23['w*S*I*t']=      G23['w'] * G23['S']* G23['sTEC'] * G23['t']
G23['w*S*I*t^2']=    G23['w'] * G23['S']* G23['sTEC'] * G23['t']**2
G23['(w)']=         G23['w'].sum()#              A4
G23['(w*I)'] = G23['w*I'].sum()
G23['(w*S)']=        G23['w*S'].sum()#      
G23['(w*S*lat)'] = G23['w*S*lat'].sum()#              A2 A3
G23['(w*S*lat^2)'] = G23['w*S*lat^2'].sum()#            A2 A3
G23['(w*S*lon)']= G23['w*S*lon'].sum()#            A2 A3
G23['(w*S*lon^2)']= (G23['w*S*lon^2']).sum()#            A2 A3
G23['(w*S*t)']= (G23['w*S*t']).sum()#            A2 A3
G23['(w*S*t^2)']= (G23['w*S*t^2']).sum()#            A2 A3
G23['w*S^2']=         G23['w'] * G23['S']**2
G23['w*S^2*lat']  =   G23['w'] * G23['S']**2 * G23['Dlat']
G23['w*S^2*lat^2']  = G23['w'] * G23['S']**2 * G23['Dlat']**2
G23['w*S^2*lat^3']  = G23['w'] * G23['S']**2 * G23['Dlat']**3
G23['w*S^2*lat^4']  = G23['w'] * G23['S']**2 * G23['Dlat']**4
G23['w*S^2*lon']  =   G23['w'] * G23['S']**2 * G23['Dlon']
G23['w*S^2*lon^2']  = G23['w'] * G23['S']**2 * G23['Dlon']**2
G23['w*S^2*lon^3']  = G23['w'] * G23['S']**2 * G23['Dlon']**3
G23['w*S^2*lon^4']  = G23['w'] * G23['S']**2 * G23['Dlon']**4
G23['w*S^2*t']  =     G23['w'] * G23['S']**2 * G23['t']
G23['w*S^2*t^2']  =   G23['w'] * G23['S']**2 * G23['t']**2
G23['w*S^2*t^3']  =   G23['w'] * G23['S']**2 * G23['t']**3
G23['w*S^2*t^4']  =   G23['w'] * G23['S']**2 * G23['t']**4
G23['w*S^2*lat*lon']  =   G23['w'] * G23['S']**2 * G23['Dlat']* G23['Dlon']
G23['w*S^2*lat^2*lon']  = G23['w'] * G23['S']**2 * G23['Dlat']**2 * G23['Dlon']
G23['w*S^2*lat*lon^2']  = G23['w'] * G23['S']**2 * G23['Dlat']* G23['Dlon']**2
G23['w*S^2*lat^2*lon^2']= G23['w'] * G23['S']**2 * G23['Dlat']**2 * G23['Dlon']**2
G23['w*S^2*lat*t']  =     G23['w'] * G23['S']**2 * G23['Dlat']* G23['t']
G23['w*S^2*lat^2*t']  =   G23['w'] * G23['S']**2 * G23['Dlat']**2 * G23['t']
G23['w*S^2*lat*t^2']  =   G23['w'] * G23['S']**2 * G23['Dlat']* G23['t']**2
G23['w*S^2*lat^2*t^2']=   G23['w'] * G23['S']**2 * G23['Dlat']**2 * G23['t']**2
G23['w*S^2*lon*t']  =     G23['w'] * G23['S']**2 * G23['Dlon']* G23['t']
G23['w*S^2*lon^2*t']  =   G23['w'] * G23['S']**2 * G23['Dlon']**2 * G23['t']
G23['w*S^2*lon*t^2']  =   G23['w'] * G23['S']**2 * G23['Dlon']* G23['t']**2
G23['w*S^2*lon^2*t^2']=   G23['w'] * G23['S']**2 * G23['Dlon']**2 * G23['t']**2
#G23['(w*S^2)']             =G23['w*S^2'].sum()               
#G23['(w*S^2*lat)']         =G23['w*S^2*lat'].sum()
#G23['(w*S^2*lat^2)']       =G23['w*S^2*lat^2'].sum()
#G23['(w*S^2*lat^3)']       =G23['w*S^2*lat^3'].sum()
#G23['(w*S^2*lat^4)']       =G23['w*S^2*lat^4'].sum()
#G23['(w*S^2*lon)']         =G23['w*S^2*lon'].sum()
#G23['(w*S^2*lon^2)']       =G23['w*S^2*lon^2'].sum()
#G23['(w*S^2*lon^3)']       =G23['w*S^2*lon^3'].sum()
#G23['(w*S^2*lon^4)']       =G23['w*S^2*lon^4'].sum()
#G23['(w*S^2*t)']           =G23['w*S^2*t'].sum() 
#G23['(w*S^2*t^2)']         =G23['w*S^2*t^2'].sum()
#G23['(w*S^2*t^3)']         =G23['w*S^2*t^3'].sum()
#G23['(w*S^2*t^4)']         =G23['w*S^2*t^4'].sum()
#G23['(w*S^2*lat*lon)']     =G23['w*S^2*lat*lon'].sum() 
#G23['(w*S^2*lat^2*lon)']   =G23['w*S^2*lat^2*lon'].sum()
#G23['(w*S^2*lat*lon^2)']   =G23['w*S^2*lat*lon^2'].sum()
#G23['(w*S^2*lat^2*lon^2)'] =G23['w*S^2*lat^2*lon^2'].sum()
#G23['(w*S^2*lat*t)']       =G23['w*S^2*lat*t'].sum() 
#G23['(w*S^2*lat^2*t)']     =G23['w*S^2*lat^2*t'].sum()
#G23['(w*S^2*lat*t^2)']     =G23['w*S^2*lat*t^2'].sum()
#G23['(w*S^2*lat^2*t^2)']   =G23['w*S^2*lat^2*t^2'].sum()
#G23['(w*S^2*lon*t)']       =G23['w*S^2*lon*t'].sum()
#G23['(w*S^2*lon^2*t)']     =G23['w*S^2*lon^2*t'].sum()
#G23['(w*S^2*lon*t^2)']     =G23['w*S^2*lon*t^2'].sum()
#G23['(w*S^2*lon^2*t^2)']   =G23['w*S^2*lon^2*t^2'].sum()  
#GG23 = G23.loc[:0,['(w*S^2)','(w*S^2*lat)','(w*S^2*lat^2)','(w*S^2*lat^3)','(w*S^2*lat^4)','(w*S^2*lon)','(w*S^2*lon^2)','(w*S^2*lon^3)','(w*S^2*lon^4)','(w*S^2*t)','(w*S^2*t^2)','(w*S^2*t^3)','(w*S^2*t^4)','(w*S^2*lat*lon)','(w*S^2*lat^2*lon)','(w*S^2*lat*lon^2)','(w*S^2*lat^2*lon^2)','(w*S^2*lat*t)','(w*S^2*lat^2*t)','(w*S^2*lat*t^2)','(w*S^2*lat^2*t^2)','(w*S^2*lon*t)','(w*S^2*lon^2*t)','(w*S^2*lon*t^2)','(w*S^2*lon^2*t^2)']]

GG = G5+G10+G13+G18+G23#     SUM j1 + j2 +j3 ... Jn  po strochno

GG['(w*S^2)'] =           GG['w*S^2'].sum()
GG['(w*S^2*lat)']  =      GG['w*S^2*lat'].sum()
GG['(w*S^2*lat^2)']  =    GG['w*S^2*lat^2'] .sum()
GG['(w*S^2*lat^3)']  =    GG['w*S^2*lat^3'].sum()
GG['(w*S^2*lat^4)']  =    GG['w*S^2*lat^4'].sum()
GG['(w*S^2*lon)']  =      GG['w*S^2*lon'].sum()
GG['(w*S^2*lon^2)']  =    GG['w*S^2*lon^2'].sum()
GG['(w*S^2*lon^3)']  =    GG['w*S^2*lon^3'].sum()
GG['(w*S^2*lon^4)']  =    GG['w*S^2*lon^4'].sum()
GG['(w*S^2*t)']  =        GG['w*S^2*t'].sum()
GG['(w*S^2*t^2)']  =      GG['w*S^2*t^2'].sum()
GG['(w*S^2*t^3)']  =      GG['w*S^2*t^3'].sum()
GG['(w*S^2*t^4)']  =      GG['w*S^2*t^4'].sum() 
GG['(w*S^2*lat*lon)']  =  GG['w*S^2*lat*lon'].sum() 
GG['(w*S^2*lat^2*lon)'] = GG['w*S^2*lat^2*lon'].sum()
GG['(w*S^2*lat*lon^2)'] = GG['w*S^2*lat*lon^2'].sum()
GG['(w*S^2*lat^2*lon^2)']=GG['w*S^2*lat^2*lon^2'].sum()
GG['(w*S^2*lat*t)']  =    GG['w*S^2*lat*t'].sum()
GG['(w*S^2*lat^2*t)']  =  GG['w*S^2*lat^2*t'].sum()  
GG['(w*S^2*lat*t^2)']   = GG['w*S^2*lat*t^2'].sum() 
GG['(w*S^2*lat^2*t^2)'] = GG['w*S^2*lat^2*t^2'].sum()
GG['(w*S^2*lon*t)']  =    GG['w*S^2*lon*t'].sum()  
GG['(w*S^2*lon^2*t)']  =  GG['w*S^2*lon^2*t'].sum()
GG['(w*S^2*lon*t^2)']  =  GG['w*S^2*lon*t^2'].sum()
GG['(w*S^2*lon^2*t^2)'] = GG['w*S^2*lon^2*t^2'].sum()

GG['(w*S*I)'] = GG['w*S*I'].sum()
GG['(w*S*I*lat)'] = GG['w*S*I*lat'].sum()
GG['(w*S*I*lat^2)'] = GG['w*S*I*lat^2'].sum()
GG['(w*S*I*lon)'] = GG['w*S*I*lon'].sum()
GG['(w*S*I*lon^2)'] = GG['w*S*I*lon^2'].sum()
GG['(w*S*I*t)'] = GG['w*S*I*t'].sum()
GG['(w*S*I*t^2)'] = GG['w*S*I*t^2'].sum()
GG1 = GG.loc[:0,['(w*S^2)','(w*S^2*lat)','(w*S^2*lat^2)','(w*S^2*lat^3)','(w*S^2*lat^4)','(w*S^2*lon)','(w*S^2*lon^2)','(w*S^2*lon^3)','(w*S^2*lon^4)','(w*S^2*t)','(w*S^2*t^2)','(w*S^2*t^3)','(w*S^2*t^4)','(w*S^2*lat*lon)','(w*S^2*lat^2*lon)','(w*S^2*lat*lon^2)','(w*S^2*lat^2*lon^2)','(w*S^2*lat*t)','(w*S^2*lat^2*t)','(w*S^2*lat*t^2)','(w*S^2*lat^2*t^2)','(w*S^2*lon*t)','(w*S^2*lon^2*t)','(w*S^2*lon*t^2)','(w*S^2*lon^2*t^2)']]
#arrData1.append(GG.values[0][1:26].tolist())
#GG1.to_csv('A_1_1.csv',sep=';')

BG1 = GG.loc[:0,['(w*S*I)','(w*S*I*lat)','(w*S*I*lat^2)','(w*S*I*lon)','(w*S*I*lon^2)','(w*S*I*t)','(w*S*I*t^2)']]
BG1.T.to_csv('B_1.csv',sep=';')

wsg5 =  G5.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
wsg10 =  G10.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
wsg13 = G13.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
#wsg15 = G15.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
wsg18 = G18.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
wsg23 = G23.loc[:0,['(w*S)','(w*S*lat)','(w*S*lat^2)','(w*S*lon)','(w*S*lon^2)','(w*S*t)','(w*S*t^2)']]
GG2 = pd.concat([wsg5,wsg10,wsg13,wsg18,wsg23])
GG2.to_csv('A_1_2.csv',sep=';')
GG2.T.to_csv('A_1_2t.csv',sep=';')

w5 =G5.loc[:0,['(w)']]
w10 =G10.loc[:0,['(w)']]
w13 =G13.loc[:0,['(w)']]
#w15 = G15.loc[:0,['(w)']]
w18 = G18.loc[:0,['(w)']]
w23 = G23.loc[:0,['(w)']]
GG_W  = pd.concat([w5,w10,w13,w18,w23])
GG_W.to_csv('A_1_3.csv',sep=';')
wI5 =G5.loc[:0,['(w*I)']]
wI10 =G10.loc[:0,['(w*I)']]
wI13 =G13.loc[:0,['(w*I)']]
#wI15 = G15.loc[:0,['(w*I)']]
wI18 = G18.loc[:0,['(w*I)']]
wI23 = G23.loc[:0,['(w*I)']]
GG_WI  = pd.concat([wI5,wI10,wI13,wI18,wI23])
#GG_WI.to_csv('B_2.csv',sep=';')

G5[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G5ji.csv',sep=';')
G10[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G10ji.csv',sep=';')
G13[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G13ji.csv',sep=';')
#G15[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G15ji.csv',sep=';')
G18[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G18ji.csv',sep=';')
G23[['w','S','Dlat','Dlon','t','sTEC']].to_csv('G23ji.csv',sep=';')
#GG['w*S^2']
GG_WI

A = GG1.values.tolist()
AA =  np.array([
    [A[0][0], A[0][1], A[0][2], A[0][5],  A[0][6],  A[0][9],  A[0][10]],
    [A[0][1], A[0][2], A[0][3], A[0][13], A[0][15], A[0][17], A[0][19]],
    [A[0][2], A[0][3], A[0][4], A[0][14], A[0][16], A[0][18], A[0][20]],
    [A[0][5], A[0][13], A[0][14], A[0][6], A[0][7], A[0][21], A[0][23]],
    [A[0][6], A[0][15], A[0][16], A[0][7], A[0][8], A[0][22], A[0][24]],
    [A[0][9], A[0][17], A[0][18], A[0][21], A[0][22], A[0][10], A[0][11]],
    [A[0][10], A[0][19], A[0][20], A[0][23], A[0][24], A[0][11], A[0][12]],
    ])

A1 = pd.DataFrame(AA, columns = ['ws','lat','lat^2','lon','lon^2','t','t^2'])
A1.to_csv('A_1.csv',sep=';')
A1
BG1.T
GG2
GG_WI

,(w*I)
0,1527.927547
0,1766.021764
0,1824.565012
0,2435.500828
0,2157.641923


In [133]:
matrixA = pd.read_csv ('a.csv',sep = ';')
matrixB = matrixA['b']
matrixA.drop(['b'], axis=1, inplace=True)

b = matrixB.values
A = matrixA.values

w = np.linalg.lstsq(A.T, b, rcond=None)[0]
w
x = pd.Series(w) 
x

0      110.397048
1      914.733399
2     5821.880448
3     -158.434836
4    -1011.629516
5       16.698863
6       49.102826
7     -119.952593
8        0.422364
9      -52.592870
10     -24.805745
11     -16.319924
dtype: float64